In [1]:
%%bash
pip3 install wordsegment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.2 MB/s eta 0:00:00


In [2]:
from google.colab import auth
import pandas as pd
import re
from wordsegment import load, segment
load()

auth.authenticate_user()

In [ ]:
# copy data from google cloud storage
!gsutil cp "gs://cil_2023/train_pos_full.txt" .
!gsutil cp "gs://cil_2023/train_neg_full.txt" .
!gsutil cp "gs://cil_2023/test_data.txt" .

Copying gs://cil_2023/train_pos_full.txt...
\ [1 files][ 82.1 MiB/ 82.1 MiB]                                                
Operation completed over 1 objects/82.1 MiB.                                     
Copying gs://cil_2023/train_neg_full.txt...
\ [1 files][101.4 MiB/101.4 MiB]                                                
Operation completed over 1 objects/101.4 MiB.                                    
Copying gs://cil_2023/test_data.txt...
/ [1 files][798.1 KiB/798.1 KiB]                                                
Operation completed over 1 objects/798.1 KiB.                                    


In [ ]:
# read data
tweets_train_pos = pd.read_fwf("train_pos_full.txt", sep='\n', header=None)[0].tolist()
tweets_train_neg = pd.read_fwf("train_neg_full.txt", sep='\n', header=None)[0].tolist()
tweets_test = pd.read_fwf("test_data.txt", sep='\n', header=None)[0].tolist()

In [3]:
def unhashtag(tweet):
  offset = 0
  for h in re.finditer(r'#\S+', tweet):
    start, end = h.span()
    new_string = ' '.join(segment(h.group(0))) + ' '
    tweet = tweet[:(start+offset)] + new_string + tweet[(end+offset+1):]
    offset += len(new_string) - (end - start + 1)
  return tweet

In [5]:
unhashtag("bla bla #ihatethis bla bla")

'bla bla i hate this bla bla'

In [ ]:
# preprocessing

# should take about 5 min
for tweetlist in [tweets_train_pos, tweets_train_neg, tweets_test]:
  for i, tweet in enumerate(tweetlist):
    if '#' in tweet:
      tweetlist[i] = unhashtag(tweet)

In [ ]:
# write preprocessed tweets back to file
for filename, tweetlist in [('train_pos_preprocessed.txt', tweets_train_pos),
                            ('train_neg_preprocessed.txt', tweets_train_neg),
                            ('test_preprocessed.txt', tweets_test)]:
  with open(filename, 'w') as f:
    for line in tweetlist:
      f.write(f"{line}\n")

In [ ]:
# copy data to google cloud storage
!gsutil cp "./train_pos_preprocessed.txt" "gs://cil_2023/"
!gsutil cp "./train_neg_preprocessed.txt" "gs://cil_2023/"
!gsutil cp "./test_preprocessed.txt" "gs://cil_2023/"

Copying file://./train_pos_preprocessed.txt [Content-Type=text/plain]...
/ [1 files][ 82.2 MiB/ 82.2 MiB]                                                
Operation completed over 1 objects/82.2 MiB.                                     
Copying file://./train_neg_preprocessed.txt [Content-Type=text/plain]...
- [1 files][101.5 MiB/101.5 MiB]                                                
Operation completed over 1 objects/101.5 MiB.                                    
Copying file://./test_preprocessed.txt [Content-Type=text/plain]...
- [1 files][798.3 KiB/798.3 KiB]                                                
Operation completed over 1 objects/798.3 KiB.                                    
